##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Temas esenciales de la ejecución "Eager"


Nota: Nuestra comunidad de Tensorflow
ha traducido estos documentos. Como las
traducciones de la comunidad son
basados en el "mejor esfuerzo", no hay
ninguna garantia que esta sea un reflejo
preciso y actual de la Documentacion
Oficial en Ingles. Si tienen sugerencias
sobre como mejorar esta traduccion, por
favor envian un "Pull request" al siguiente
repositorio tensorflow/docs. Para
ofrecerse como voluntario o hacer
revision de las traducciones de la
Comunidad por favor contacten al
siguiente grupo docs@tensorflow.org list.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/eager"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Visualizar en TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/eager.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Ejecutar en Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/eager.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Ver código fuente en GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/eager.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Descargar notebook</a>
  </td>
</table>


"Eager" es un entorno de programación imperativo que
evalúa las operaciones de inmediato, sin construir grafos: las operaciones regresan
valores concretos en lugar de construir un grafo computacional para ejecutar más tarde. Esto
facilita comenzar con los modelos TensorFlow,depuración, y
reduce el  codigo repetitivo. Para seguir esta guía, ejecute el código.
ejemplos a continuación en un intérprete interactivo de `python`.

La ejecución "eager" es una plataforma flexible de aprendizaje automático para la investigación y
experimentación, proporcionando:

* * Una interfaz intuitiva *: estructura tu código de forma natural y usa datos de Python
  estructuras Repita rápidamente en modelos pequeños y datos pequeños.
* * Depuración más fácil *: llame a las operaciones directamente para inspeccionar los modelos en ejecución y probar
  cambios Utilice las herramientas de depuración de Python estándar para la generación de informes de errores inmediatos.
* * Flujo de control natural *: utilice el flujo de control de Python en lugar del control de gráfico
  flujo, simplificando la especificación de modelos dinámicos.

La ejecución rápida admite la mayoría de las operaciones de TensorFlow y la aceleración de GPU.

Nota: Algunos modelos pueden experimentar una sobrecarga mayor con una ejecución "eager"
habilitado Las mejoras de rendimiento están en curso, pero por favor
[presentar un error] (https://github.com/tensorflow/tensorflow/issues) si encuentra un
problema y comparte tu solución.

## Preparación y uso básico

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x  #gpu
except Exception:
  pass
import tensorflow as tf

import cProfile

En Tensorflow 2.0, la ejecución eager está habilitada por defecto.

In [0]:
tf.executing_eagerly()

Ahora puedes ejecutar operaciones en TensorFlow y los resultados se ejecutarán inmediatamente:

In [0]:
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))

La habilitación de la ejecución "eager" cambia la forma en que se comportan las operaciones de TensorFlow; ahora evalúan de inmediato y devuelven sus valores a Python. los objetos hacen referencia a valores concretos en lugar de identificadores simbólicos a nodos en un grafo. Dado que no hay un grafo computacional para construir y ejecutar más tarde en una sesión, es fácil inspeccionar los resultados usando `print ()` o un depurador. Evaluar, imprimir y verificar los valores del tensor no interrumpe el flujo para calcular gradientes.

La ejecución "eager" funciona bien con [NumPy](http://www.numpy.org/). Las operaciones de NumPy aceptan argumentos de `tf.Tensor`. Las [operaciones matemáticas](https://www.tensorflow.org/api_guides/python/math_ops) TensorFlow convierten los objetos Python y las matrices NumPy en objetos `tf.Tensor`. El método `tf.Tensor`. los métodos de numpy devuelven el valor del objeto como una matriz de datos (`ndarray`) NumPy.


In [0]:
a = tf.constant([[1, 2],
                 [3, 4]])
print(a)

In [0]:
# Soporte de broadcasting
b = tf.add(a, 1)
print(b)

In [0]:
# La sobrecarga de operadores es compatible
print(a * b)

In [0]:
# Usar funciones de NumPy
import numpy as np

c = np.multiply(a, b)
print(c)

In [0]:
# Obtener valores numpy de un tensor:
print(a.numpy())
# => [[1 2]
#     [3 4]]

## Flujo de control dinámico

Una ventaja importante de la ejecución "eager" es que toda la funcionalidad del host el idioma está disponible mientras su modelo se está ejecutando. Así por ejemplo,
es facil de escribir [fizzbuzz](https://en.wikipedia.org/wiki/Fizz_buzz):

In [0]:
def fizzbuzz(max_num):
  counter = tf.constant(0)
  max_num = tf.convert_to_tensor(max_num)
  for num in range(1, max_num.numpy()+1):
    num = tf.constant(num)
    if int(num % 3) == 0 and int(num % 5) == 0:
      print('FizzBuzz')
    elif int(num % 3) == 0:
      print('Fizz')
    elif int(num % 5) == 0:
      print('Buzz')
    else:
      print(num.numpy())
    counter += 1

In [0]:
fizzbuzz(15)

Esto tiene condicionales que dependen de los valores del tensor e imprime estos valores en tiempo de ejecución.

## Ejercicios utilizando ejercución "Eager"

### Computación de gradientes

[Automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation)
es útil para implementar con algoritmos de aprendizaje automático como
[backpropagation](https://en.wikipedia.org/wiki/Backpropagation) para entrenar redes neuronales. Durante la ejecución "eager", usa `tf.GradientTape` para rastrear operaciones y calcular gradientes más tarde.

Puede usar `tf.GradientTape` para entrenar y / o calcular gradientes con "eager". Es especialmente útil para bucles complicados de entrenamiento.

Dado que pueden ocurrir diferentes operaciones durante cada llamada, todos
las operaciones de pase directo se graban en una "cinta". Para calcular el gradiente, juega la cinta al revés y luego descartar. Un `tf.GradientTape` particular solo puede
calcular un gradiente; llamadas posteriores arrojan un error de tiempo de ejecución.

In [0]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
  loss = w * w

grad = tape.gradient(loss, w)
print(grad)  # => tf.Tensor([[ 2.]], shape=(1, 1), dtype=float32)

### Entrenar un modelo

El siguiente ejemplo crea un modelo de varias capas que clasifica los dígitos manuscritos MNIST estándar. Demuestra el optimizador y las API de capa para crear grafos entrenables en un entorno de ejecución ansioso.

In [0]:
# Obtener y dar forma a los datos mnist
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices(
  (tf.cast(mnist_images[...,tf.newaxis]/255, tf.float32),
   tf.cast(mnist_labels,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [0]:
# Construir el modelo
mnist_model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16,[3,3], activation='relu',
                         input_shape=(None, None, 1)),
  tf.keras.layers.Conv2D(16,[3,3], activation='relu'),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10)
])


Incluso sin capacitación, llame al modelo e inspeccione la salida en ejecución "eager":

In [0]:
for images,labels in dataset.take(1):
  print("Logits: ", mnist_model(images[0:1]).numpy())

Mientras que los modelos keras tienen un ciclo de entrenamiento incorporado (usando el método `fit`), a veces necesitas más personalización. Aquí hay un ejemplo de un ciclo de entrenamiento implementando "eager":

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

loss_history = []

Nota: Use las funciones de aserción en `tf.debugging` para verificar si una condición se mantiene. Esto funciona en la ejecución "eager" y la ejecución del grafo.

In [0]:
def train_step(images, labels):
  with tf.GradientTape() as tape:
    logits = mnist_model(images, training=True)
    
    # Agregar "asserts" para verificar la forma de la salida.
    tf.debugging.assert_equal(logits.shape, (32, 10))
    
    loss_value = loss_object(labels, logits)

  loss_history.append(loss_value.numpy().mean())
  grads = tape.gradient(loss_value, mnist_model.trainable_variables)
  optimizer.apply_gradients(zip(grads, mnist_model.trainable_variables))

In [0]:
def train():
  for epoch in range(3):
    for (batch, (images, labels)) in enumerate(dataset):
      train_step(images, labels)
    print ('Epoch {} finished'.format(epoch))

In [0]:
train()

In [0]:
import matplotlib.pyplot as plt

plt.plot(loss_history)
plt.xlabel('Batch #')
plt.ylabel('Loss [entropy]')

### Variables y optimizadores

Los objetos `tf.Variable` almacenan valores mutables de` tf.Tensor` a los que se accede durante capacitación para facilitar la diferenciación automática. Los parámetros de un modelo pueden ser encapsulado en clases como variables.

Es mejor encapsular los parámetros del modelo utilizando `tf.Variable` con
`tf.GradientTape`. Por ejemplo, el ejemplo de diferenciación automática anterior
puede reescribirse:


In [0]:
class Model(tf.keras.Model):
  def __init__(self):
    super(Model, self).__init__()
    self.W = tf.Variable(5., name='weight')
    self.B = tf.Variable(10., name='bias')
  def call(self, inputs):
    return inputs * self.W + self.B

# Un conjunto de datos de puntos alrededor 3 * x + 2
NUM_EXAMPLES = 2000
training_inputs = tf.random.normal([NUM_EXAMPLES])
noise = tf.random.normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

# La función de pérdida a optimizar
def loss(model, inputs, targets):
  error = model(inputs) - targets
  return tf.reduce_mean(tf.square(error))

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, [model.W, model.B])

# Definir:
# 1. Un modelo.
# 2. Derivadas de una función de pérdida con respecto a los parámetros del modelo.
# 3. Una estrategia para actualizar las variables basadas en los derivados.
model = Model()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

print("Initial loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))

# Training loop
for i in range(300):
  grads = grad(model, training_inputs, training_outputs)
  optimizer.apply_gradients(zip(grads, [model.W, model.B]))
  if i % 20 == 0:
    print("Loss at step {:03d}: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))

print("Final loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))
print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))

## Use objetos para el estado durante la ejecución "eager"

Con la ejecución del grado TF 1.x, el estado del programa (como las variables) se almacena en colecciones globales y su vida útil es administrada por el objeto `tf.Session`. Por el contrario, durante la ejecución ansiosa, la vida útil de los objetos de estado está determinada por la vida útil de su objeto Python correspondiente.

### Las variables son objetos

Durante la ejecución "eager", las variables persisten hasta que se elimina la última referencia al objeto y luego se elimina.

In [0]:
if tf.config.experimental.list_physical_devices("GPU"):
  with tf.device("gpu:0"):
    print("GPU enabled")
    v = tf.Variable(tf.random.normal([1000, 1000]))
    v = None  # v no longer takes up GPU memory

### Guardado basado en objetos

Esta sección es una versión abreviada de [guía de puntos de control de entrenamiento](./checkpoints.ipynb).

`tf.train.Checkpoint` puede guardar y restaurar` tf.Variable` y crear "checkpoints":

In [0]:
x = tf.Variable(10.)
checkpoint = tf.train.Checkpoint(x=x)

In [0]:
x.assign(2.)   # Asigne un nuevo valor a las variables y guarde.
checkpoint_path = './ckpt/'
checkpoint.save('./ckpt/')

In [0]:
x.assign(11.)  # Cambiar la variable después de guardar.

# Restaurar valores desde el punto de control
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))

print(x)  # => 2.0

Para guardar y cargar modelos, `tf.train.Checkpoint` almacena el estado interno de los objetos, sin requerir variables ocultas. Para registrar el estado de un `modelo`, un` optimizador` y un paso global, páselos a un `tf.train.Checkpoint`:

In [0]:
import os

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16,[3,3], activation='relu'),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10)
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
checkpoint_dir = 'path/to/model_dir'
if not os.path.exists(checkpoint_dir):
  os.makedirs(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
root = tf.train.Checkpoint(optimizer=optimizer,
                           model=model)

root.save(checkpoint_prefix)
root.restore(tf.train.latest_checkpoint(checkpoint_dir))

Nota: En muchos bucles de entrenamiento, las variables se crean después de llamar a `tf.train.Checkpoint.restore`. Estas variables se restaurarán tan pronto como se creen, y las aserciones están disponibles para garantizar que se haya cargado completamente un punto de control. Ver el [guía de puntos de control de entrenamiento](./checkpoints.ipynb) para más detalles.

### Métricas orientadas a objetos

`tf.keras.metrics` se almacenan como objetos. Actualizar una métrica pasando los nuevos datos al invocable y recupere el resultado utilizando el método `tf.keras.metrics.result`, por ejemplo:

In [0]:
m = tf.keras.metrics.Mean("loss")
m(0)
m(5)
m.result()  # => 2.5
m([8, 9])
m.result()  # => 5.5

## Temas avanzados de diferenciación automática

### Modelos dinámicos

`tf.GradientTape` también se puede utilizar en modelos dinámicos. Este ejemplo para un
[backtracking line search](https://wikipedia.org/wiki/Backtracking_line_search)
El algoritmo se parece al código NumPy, excepto que hay gradientes y es diferenciable, a pesar del flujo de control complejo:

In [0]:
def line_search_step(fn, init_x, rate=1.0):
  with tf.GradientTape() as tape:
    # Las variables se registran automáticamente, pero observan manualmente un tensor
    tape.watch(init_x)
    value = fn(init_x)
  grad = tape.gradient(value, init_x)
  grad_norm = tf.reduce_sum(grad * grad)
  init_value = value
  while value > init_value - rate * grad_norm:
    x = init_x - rate * grad
    value = fn(x)
    rate /= 2.0
  return x, value

### Gradientes personalizados

Los gradientes personalizados son una manera fácil de sobreescribir los gradientes. Dentro de la función de avance, defina el gradiente con respecto a entradas, salidas o resultados intermedios. Por ejemplo, aquí hay una manera fácil de recortar La norma de los gradientes en el paso hacia atrás:

In [0]:
@tf.custom_gradient
def clip_gradient_by_norm(x, norm):
  y = tf.identity(x)
  def grad_fn(dresult):
    return [tf.clip_by_norm(dresult, norm), None]
  return y, grad_fn

Los gradientes personalizados se usan comúnmente para proporcionar un gradiente numéricamente estable para un secuencia de operaciones:

In [0]:
def log1pexp(x):
  return tf.math.log(1 + tf.exp(x))

def grad_log1pexp(x):
  with tf.GradientTape() as tape:
    tape.watch(x)
    value = log1pexp(x)
  return tape.gradient(value, x)


In [0]:
# El cálculo de gradiente funciona bien en x = 0.
grad_log1pexp(tf.constant(0.)).numpy()

In [0]:
# Sin embargo, x = 100 falla debido a la inestabilidad numérica.

grad_log1pexp(tf.constant(100.)).numpy()

Aquí, la función `log1pexp` se puede simplificar analíticamente con un gradiente personalizado. La siguiente implementación reutiliza el valor de `tf.exp (x)` que se calcula durante el paso directo, lo que lo hace más eficiente al eliminar los cálculos redundantes

In [0]:
@tf.custom_gradient
def log1pexp(x):
  e = tf.exp(x)
  def grad(dy):
    return dy * (1 - 1 / (1 + e))
  return tf.math.log(1 + e), grad

def grad_log1pexp(x):
  with tf.GradientTape() as tape:
    tape.watch(x)
    value = log1pexp(x)
  return tape.gradient(value, x)


In [0]:
# Como antes, el cálculo del gradiente funciona bien en x = 0.
grad_log1pexp(tf.constant(0.)).numpy()

In [0]:
# Y el cálculo de gradiente también funciona en x = 100.
grad_log1pexp(tf.constant(100.)).numpy()

## Performance

La computación se descarga automáticamente a las GPU durante la ejecución ansiosa. Si desea controlar dónde se ejecuta un cálculo, puede encerrarlo en un bloque `tf.device ('/ gpu: 0')` (o el equivalente de la CPU):

In [0]:
import time

def measure(x, steps):
  # TensorFlow inicializa una GPU la primera vez que se usa, excluye el tiempo.
  tf.matmul(x, x)
  start = time.time()
  for i in range(steps):
    x = tf.matmul(x, x)
  # tf.matmul puede regresar antes de completar la multiplicación de la matriz
  # (p. ej., puede regresar después de aplicar la operación en una secuencia CUDA).
  # La llamada x.numpy () a continuación asegurará que todas las operaciones en cola
  # ha completado (y también copiará el resultado a la memoria del host,
  # así que estamos incluyendo un poco más que solo la operación matmul
  # hora).
  _ = x.numpy()
  end = time.time()
  return end - start

shape = (1000, 1000)
steps = 200
print("Time to multiply a {} matrix by itself {} times:".format(shape, steps))

# Se ejecuta en CPU:
with tf.device("/cpu:0"):
  print("CPU: {} secs".format(measure(tf.random.normal(shape), steps)))

# Se ejecuta en GPU, si esta disponible:
if tf.config.experimental.list_physical_devices("GPU"):
  with tf.device("/gpu:0"):
    print("GPU: {} secs".format(measure(tf.random.normal(shape), steps)))
else:
  print("GPU: not found")

A `tf.Tensor` object can be copied to a different device to execute its
operations:

In [0]:
if tf.config.experimental.list_physical_devices("GPU"):
  x = tf.random.normal([10, 10])

  x_gpu0 = x.gpu()
  x_cpu = x.cpu()

  _ = tf.matmul(x_cpu, x_cpu)    # Se ejecuta en CPU
  _ = tf.matmul(x_gpu0, x_gpu0)  # Se ejecuta en GPU:0

### Benchmarks

Para modelos pesados en cómputo, como
[ResNet50](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/eager/python/examples/resnet50)
entrenamiento en una GPU, el rendimiento de ejecución "eager" es comparable a la ejecución `tf.function`.
Pero esta brecha crece para modelos con menos cómputo y hay trabajo para
debe hacerse para optimizar rutas de código activo para modelos con muchas operaciones pequeñas.

## Trabajar con funciones

Si bien la ejecución "eager" hace que el desarrollo y la depuración sean más interactivos, La ejecución de grafos de estilo TensorFlow 1.x tiene ventajas para la capacitación distribuida, el rendimiento optimizaciones e implementación de producción. Para cerrar esta brecha, TensorFlow 2.0 introduce `function`s a través de la API` tf.function`. Para más información, vea la guía [Autograph](./autograph.ipynb).